In [1]:
from math import ceil
import random

# Simple Tabulation Hash

In [2]:
def particionar(x):
    binario_x = bin(x).zfill(64).replace('b','')

    particoes = []
    for _ in range(8):                   #Aqui utilizo c=8.
        particoes.append(binario_x[-8:]) #Pego os 8 ultimos e adiciono a partição.
        binario_x = binario_x[:-8]       #Atualizo o binario_x retirando os 8 últimos pra continuar a iteração até que não
                                         #sobrem mais caracteres.
    return particoes[::-1]               #Inverto a lista para que os últimos caracteres da repres. binária vão para as
                                         #as últimas partições.

In [3]:
def tabela_aleatoria(): #Aqui teremos 8 tabelas totalmente aleatórias que armazenam 2**c=256 elementos.
    T = []
    random.seed(10)
    for __ in range(8):
        T_i = []
        for _ in range(256):
            valor_aleatorio = random.randint(0,256)
            T_i.append(valor_aleatorio)
        T.append(T_i)
    return T

In [4]:
def funcao_hash(x): #h(x) = T[1](x[1])^T[2](x[2])^...^T[8]x[8]
    particoes = particionar(int(x))
    particao_decimal = [int(particao, 2) for particao in particoes]
    
    T = tabela_aleatoria()
    T_j = []
    for i in range(8):
        j = particao_decimal[i]
        T_j.append(T[i][j])
    
    h_x = T_j[0]^T_j[1]
    for i in range(2,8):
        h_x = h_x^T_j[i]
        
    return h_x

# Funções Auxiliares

In [5]:
def inicializar_tabela(m):
    return [None]*m

In [6]:
def limpar(tabela):
    return 0

In [7]:
def remapear_valores(tabela_antiga, tabela):
    for pos, valor in enumerate(tabela_antiga):
        if(valor != None and valor != '#'): # '#' indica que foi removido um elemento ali
            incluir(tabela, valor)
    return tabela

In [8]:
def table_doubling(tabela):
    tabela_antiga = tabela.copy()
    m = len(tabela)
    m *= 2
    tabela = inicializar_tabela(m)
    tabela = remapear_valores(tabela_antiga, tabela)
    
    return tabela

In [9]:
def halving(tabela):
    tabela_antiga = tabela.copy()
    m = len(tabela)
    m = ceil(m/2)
    tabela = inicializar_tabela(m)
    tabela = remapear_valores(tabela_antiga, tabela)
    
    return tabela

# Operações Hash

In [10]:
def incluir(tabela_hash, x):
    global log_operacoes
    global contador_elementos
    
    h_x = funcao_hash(x)
    inserido = False
    i=0
    while(not inserido):
        if(i<len(tabela_hash)):            #pra não percorrer a tabela infinitas vezes.
            pos = (h_x+i)%len(tabela_hash) #(h[x]+i)mod m
            if(tabela_hash[pos] == None or tabela_hash[pos] == '#'): # '#' indica que foi removido um elemento ali
                tabela_hash[pos] = x
                inserido = True
                log_operacoes.append("INC:{}\n{} {}\n\n".format(x, h_x%len(tabela_hash), pos))
            else:
                i+=1
        else:
            tabela_hash = table_doubling(tabela_hash) #se a tabela ja estiver cheia, table doubling remapeando valores.
            log_operacoes.append("DOBRAR TAM:{}\n\n".format(len(tabela_hash)))
            incluir(tabela_hash, x)
    
    contador_elementos+=1
    
    return tabela_hash

In [11]:
def buscar(tabela, x):
    h_x = funcao_hash(x)
    encontrou = False
    i=0
    
    while(not encontrou):
        if(i<len(tabela)):            #Pra não percorrer a tabela infinitas vezes.
            pos = (h_x+i)%len(tabela) #(h[x]+i)mod m
            if(tabela[pos] == x):          #Caso eu encontre o elemento, retorno sua posição.
                return pos
            else:
                i+=1
        else:
            encontrou = True
    return -1                              #Caso o elemento não for encontrado, retorno -1.

In [12]:
def remover(tabela, x): #Implementar o halving (done)
    global contador_elementos
    global log_operacoes
    
    h_x = funcao_hash(x)
    pos = buscar(tabela, x) 
    if(pos != -1):
        tabela[pos] = '#'
        contador_elementos-=1
        #print("Contador: ",contador_elementos)
        log_operacoes.append("REM:{}\n{} {}\n\n".format(x, h_x%len(tabela), pos))
        
        #Fazer o halving quando só 1/4 da tabela estiver preenchida e tabela com tamanho pelo menos (1+epsilon)n.
        if(contador_elementos<=(1/4)*len(tabela)): #and (len(tabela)/2)>=(1+1)): 
            tabela = halving(tabela)
            log_operacoes.append("METADE TAM:{}\n\n".format(len(tabela)))
        
    return tabela

# Execução

In [16]:
contador_elementos = 0
log_operacoes = []

In [17]:
def executar():
    global log_operacoes
    
    tabela_hash = inicializar_tabela(8)
    arquivo = open('entrada.txt','r')
    saida = open('saida.txt','w')
    
    log_operacoes.append("TAM:{}\n".format(len(tabela_hash)))
    for line in arquivo:
        operacao, x = line.split(':')
        if(operacao == 'INC'):
            incluir(tabela_hash, int(x))

        elif(operacao == 'REM'):
            remover(tabela_hash, int(x))

        elif(operacao == 'BUS'):
            buscar(tabela_hash, int(x))
            log_operacoes.append("BUS:{}{} {}\n\n".format(x, funcao_hash(x)%len(tabela_hash), buscar(tabela_hash,x)))
    
    saida.writelines(log_operacoes)
    
    arquivo.close()
    saida.close()

In [18]:
executar()